In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict

import praw

import json

from datetime import datetime

pd.options.display.max_colwidth = 75

In [2]:
df = pd.read_csv('../data/movies_cleaned.csv', index_col='id')

In [3]:
with open('../.secret/ZSDSFI_client_id.txt') as f:
    client_id = f.read()

with open('../.secret/ZSDSFI_client_secret.txt') as f:
    client_secret = f.read()

In [4]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="Movie Scraper by ZSDSFI"
)

In [5]:
df.head()

,title,post_title,url
id,,,
vzcwal,the princess,official discussion - the princess [spoilers],https://www.reddit.com/r/movies/comments/vzcwal/official_discussion_the...
vzcw0a,the man from toronto,official discussion - the man from toronto [spoilers],https://www.reddit.com/r/movies/comments/vzcw0a/official_discussion_the...
vzcvsd,the sea beast,official discussion - the sea beast [spoilers],https://www.reddit.com/r/movies/comments/vzcvsd/official_discussion_the...
vzcvkz,mrs. harris goes to paris,official discussion - mrs. harris goes to paris [spoilers],https://www.reddit.com/r/movies/comments/vzcvkz/official_discussion_mrs...
vzcv66,where the crawdads sing,official discussion - where the crawdads sing [spoilers],https://www.reddit.com/r/movies/comments/vzcv66/official_discussion_whe...


In [6]:
all_ids = df.index.tolist()

In [7]:
# The following cell can over one hour to run.

In [8]:
movies_comments = {}

for id in all_ids:
    submission = reddit.submission(id=id)
    comments = []
    # Using a try-except block because at least one movie causes an error, as the thread no longer exists 
    try:
        submission.comment_sort = 'top'
        post_date_utc = submission.created_utc
        for top_level_comment in submission.comments[:100]:
            comments.append(top_level_comment.body)
    except:
        continue
    movies_comments[id] = {
        'title': df.loc[id]['title'],
        'comments': comments,
        'post_date_utc': post_date_utc
    }

In [27]:
pd.DataFrame(movies_comments_dict).T.info()

<class 'pandas.core.frame.DataFrame'>
Index: 944 entries, vzcwal to 47szbr
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          944 non-null    object
 1   comments       944 non-null    object
 2   post_date_utc  944 non-null    object
dtypes: object(3)
memory usage: 29.5+ KB


In [28]:
pd.DataFrame(movies_comments_dict).T.sample(10)

,title,comments,post_date_utc
riy7d4,red rocket,"[This was such a beautiful and human movie, and I expected nothing less...",1639797824.0
k4c57u,superintelligence,[honestly this movie would be a solid 6-7/10 for me if they had gotten ...,1606791751.0
9x2e23,fantastic beasts: the crimes of grindelwald,"[Did Grindelwald just vape World War 2?, Queenies motivations made zero...",1542330028.0
ej1r13,the grudge (2020),"[Well folks, looks like we got our first **FUCK YOU, IT'S JANUARY!** re...",1578020426.0
nsj1ko,sailor moon eternal,"[So ill be honest, i know almost nothing about sailor moon, but last ni...",1622851352.0
bzclyb,rolling thunder revue: a bob dylan story by martin scorsese,"[So, I feel like there needs to be an official accounting of which part...",1560331824.0
a6qtnh,welcome to marwen,[Was this movie made for a five year old? The audacity they have to ass...,1545361223.0
sw0jcl,texas chainsaw massacre (2022),[Sally trained for like 50 years to kill Leatherface yet came to confro...,1645242013.0
790a25,suburbicon,"[What a disorganized, unfulfilling mess. It just felt like Great Value ...",1509069290.0
81bwds,red sparrow,[Sat through entire movie.\nAs a bird watcher I didn’t not see what I e...,1519966853.0


In [29]:
# Saving the lists of movie comments as a JSON file

with open("../data/movies_comments.json", "w") as outfile:
    json.dump(movies_comments, outfile)